<a href="https://colab.research.google.com/github/mashdas/nlp_101/blob/master/shitty_poem_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2020-05-21 09:41:05--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 2607:f8b0:400c:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2020-05-21 09:41:05 (111 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [0]:
tk=Tokenizer()

data=open('/tmp/irish-lyrics-eof.txt').read()

stuff=data.lower().split("\n")

In [0]:
stuff

In [0]:
tk.fit_on_texts(stuff)

In [0]:
word_index=tk.word_index

In [21]:
total_words=len(word_index)+1##For that one outer vocabulary token
total_words

2690

In [0]:
input_seq=[]
for i in stuff:
  holder=tk.texts_to_sequences([i])
  n_gram_seq=[holder[0][0:x+2] for x in range(len(holder[0])-1)]
  input_seq.append(n_gram_seq)



In [0]:
max_sequence_len=max([len(x) for x in input_seq])

In [0]:
# ##PADDING THEM INPUTS

input_seq
ip_seq=[]
for x in input_seq:
  for xi in x:
    ip_seq.append(xi)

In [0]:
import numpy as np
input_sequences=np.array(pad_sequences(ip_seq,maxlen=max_sequence_len,padding='pre'))

In [0]:
#Create ips and labels
xs,labels=input_sequences[:,:-1],input_sequences[:,-1]
ys=tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [115]:
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,activation="softmax"))
model.compile(loss="categorical_crossentropy",optimizer=Adam(lr=0.01),metrics=['accuracy'])
history=model.fit(xs,ys,epochs=100,verbose=1)

Epoch 1/100
377/377 [==============================] - 22s 58ms/step - loss: 6.6418 - accuracy: 0.0743
Epoch 2/100
377/377 [==============================] - 21s 57ms/step - loss: 5.7399 - accuracy: 0.1134
Epoch 3/100
377/377 [==============================] - 21s 57ms/step - loss: 4.9103 - accuracy: 0.1597
Epoch 4/100
377/377 [==============================] - 21s 57ms/step - loss: 4.0120 - accuracy: 0.2339
Epoch 5/100
377/377 [==============================] - 22s 57ms/step - loss: 3.1978 - accuracy: 0.3270
Epoch 6/100
377/377 [==============================] - 22s 57ms/step - loss: 2.4863 - accuracy: 0.4448
Epoch 7/100
377/377 [==============================] - 21s 57ms/step - loss: 2.0196 - accuracy: 0.5295
Epoch 8/100
377/377 [==============================] - 22s 57ms/step - loss: 1.6091 - accuracy: 0.6192
Epoch 9/100
377/377 [==============================] - 22s 57ms/step - loss: 1.3664 - accuracy: 0.6697
Epoch 10/100
377/377 [==============================] - 22s 58ms/step - l

In [123]:
seed_text = "My name is Jeff"
next_words = 20
  
for _ in range(next_words):
	token_list = tk.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tk.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

My name is Jeff soothed in solace and much weeping and pain said unseen more meet now now me fists so now now now
